Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

`url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'`

En este caso os recomendamos que uséis el producto `meteo` para obtener la información climática. Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el *dataset*.

Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. Pero antes de poneros manos a la obra, tenemos muchísimos países y esto puede hacerse eterno. Sacad la información climática solo para los siguientes países, con las siguientes coordenadas:

Pais, Latitud, Longitud

USA,
39.7837304,
-100.445882

Australia,
-24.7761086,
134.755

South Africa,
-28.8166236,
24.991639

New Zealand,
-41.5000831,
172.8344077

Papua New Guinea,
-5.6816069,
144.2489081

* Requisitos de este ejercicio de pair programming:
* Deberéis meter toda la información climática en un único dataframe.
* Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que iterar por la lista de países y sacar la información del clima para cada uno de ellos.
* Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del país.
* Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡:

* Crearos un diccionario donde:
    * Las keys sean los nombres de los paises

    * Los values sean las coordenadas de los paises en formato lista.

* Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura:
```python 
for key, value in diccionario.items():
    print(key, value)
```
* Crearnos un dataframe vacío donde iremos concatenando los datos obtenidos de la API cada país.

* Guardad el csv ya que mañana lo necesitaremos para hacer el ejercicio de pair programming.

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
# Definimos el diccionario
diccionario = {'USA':[39.7837304, -100.445882], 
               'Australia':[-24.7761086, 134.755], 
               'South Africa':[-28.8166236, 24.991639], 
               'New Zealand':[-41.5000831, 172.8344077], 
               'Papua New Guinea':[-5.6816069, 144.2489081]}

In [4]:
lon = -100.445882
lat = 39.7837304
producto = 'meteo'
url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

In [21]:
response = requests.get(url)
df = pd.json_normalize(response.json()['dataseries'])

In [22]:
df.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 235, 'speed':...",24,-1,11,1007,rain,2,0,290,2
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 275, 'speed':...",24,-4,10,1008,rain,3,0,300,2
2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",26,-4,7,1009,none,3,0,220,3
3,12,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",28,-1,3,1009,none,3,0,210,3
4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",29,-1,2,1008,none,3,0,185,3


In [13]:
# Hacemos el bucle for

def extracciones_paises(diccionario):

    df_all= pd.DataFrame()
    
    for key, value in diccionario.items():
        # Extraemos la latitud y la longitud
        lat = value[0]
        lon = value[1]
        pais = key
        producto = 'meteo'

        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        # Llamamos a la Api para cada dataframe vacío
        response = requests.get(url)
     
        df = pd.json_normalize(response.json()['dataseries'])
        df['pais'] = key
        df['latitud'] = lat
        df['longitud'] = lon
        df['fecha'] = datetime.now()  

        df_all = pd.concat([df, df_all], axis = 0)

    if response.status_code == 200:
        print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',response.status_code,' y como razón del código de estado: ',response.reason)
    elif response.status_code == 402:
        print('No se ha podido autorizar usario, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
    elif response.status_code == 404:
        print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
    else:
        print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
        
    return df_all
    

In [14]:
df_paises = extracciones_paises(diccionario)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [15]:
df_paises.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,fecha
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 115, 'speed':...",26,6,9,1011,none,0,0,110,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 115, 'speed':...",26,6,9,1014,none,0,0,110,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455
2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 120, 'speed':...",25,6,9,1014,none,0,0,115,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455
3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 120, 'speed':...",25,2,9,1012,none,0,0,115,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455
4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 105, 'speed':...",25,2,9,1013,none,0,0,105,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455


In [16]:
df_paises.to_csv('datos/datos_clima_paises.csv')